In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import time
import random

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import SVC

NUM_OF_ROWS = 25000
ATTACK_NAME = 'DoS'

---

In [18]:
# import the attack sample dataset
dosSamples = pd.read_csv('dos_samples.csv')
dosSamples

,Number of Ports,Average Packet Size,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,...,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Total,IAT Max,IAT Mean,IAT Std
0,1,215.800000,0,896,215.800000,342.577524,1.173594e+05,98.0,98,49.000000,...,0.0,0,5,0,0.244111,20.482480,0.244111,0.190749,0.061028,0.077588
1,5,0.000000,0,0,0.000000,0.000000,0.000000e+00,0.0,0,0.000000,...,0.0,0,0,0,0.185978,53.769819,0.185978,0.149762,0.020664,0.045901
2,1,580.041667,0,6284,580.041667,1301.573486,1.694094e+06,6659.0,1460,605.363636,...,0.0,2,23,0,0.315896,75.974420,0.315896,0.063985,0.013735,0.025389
3,1,459.100000,0,4262,459.100000,1268.255136,1.608471e+06,137.0,98,27.400000,...,0.0,0,10,0,0.887799,11.263811,0.887799,0.318239,0.098644,0.129379
4,14,118.690323,0,1460,118.690323,234.713898,5.509061e+04,11115.0,1460,140.696203,...,0.0,27,141,0,0.650434,238.302419,0.650434,0.514353,0.004224,0.041614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,1,539.515152,22,1230,539.515152,555.354275,3.084184e+05,10483.0,1230,655.187500,...,0.0,0,0,0,0.194769,169.431562,0.194769,0.032693,0.006087,0.010813
324,1,442.769231,0,1220,442.769231,563.748465,3.178123e+05,11438.0,1220,817.000000,...,0.0,2,25,0,0.116580,223.022799,0.116580,0.051589,0.004663,0.013088
325,0,0.000000,0,0,0.000000,0.000000,0.000000e+00,0.0,0,0.000000,...,0.0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
326,0,0.000000,0,0,0.000000,0.000000,0.000000e+00,0.0,0,0.000000,...,0.0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [19]:
# get all the relevant attack rows from the attack sample dataset
dosSamples = dosSamples[dosSamples['SYN Flag Count'] >= 50]

In [20]:
# print some general information about the attack samples
print(f'Dataset Shape: {dosSamples.shape}')
dosSamples.reset_index()

Dataset Shape: (32, 26)


,index,Number of Ports,Average Packet Size,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,...,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Total,IAT Max,IAT Mean,IAT Std
0,8,1,0.0,0,0,0.0,0.0,0.0,0.0,0,...,0.0,9789,0,0,0.148502,65918.254666,0.148502,0.003196,0.000015,0.000089
1,9,1,0.0,0,0,0.0,0.0,0.0,0.0,0,...,0.0,10000,0,0,0.152725,65477.172853,0.152725,0.003479,0.000015,0.000092
2,11,1,0.0,0,0,0.0,0.0,0.0,0.0,0,...,0.0,9999,0,0,0.141355,70736.778565,0.141355,0.008787,0.000014,0.000120
3,12,1,0.0,0,0,0.0,0.0,0.0,0.0,0,...,0.0,10000,0,0,0.149577,66855.135390,0.149577,0.014543,0.000015,0.000169
4,25,1,0.0,0,0,0.0,0.0,0.0,0.0,0,...,0.0,9792,0,0,0.122411,79992.802837,0.122411,0.001845,0.000013,0.000076
5,27,1,0.0,0,0,0.0,0.0,0.0,0.0,0,...,0.0,9996,0,0,0.171828,58174.442845,0.171828,0.005058,0.000017,0.000131
6,31,1,0.0,0,0,0.0,0.0,0.0,0.0,0,...,0.0,9987,0,0,0.141910,70375.670639,0.141910,0.003594,0.000014,0.000084
7,32,1,0.0,0,0,0.0,0.0,0.0,0.0,0,...,0.0,10000,0,0,0.103396,96715.597430,0.103396,0.003315,0.000010,0.000078
8,34,1,0.0,0,0,0.0,0.0,0.0,0.0,0,...,0.0,9999,0,0,0.114260,87510.971924,0.114260,0.002452,0.000011,0.000074
9,38,1,0.0,0,0,0.0,0.0,0.0,0.0,0,...,0.0,9995,0,0,0.161397,61927.956877,0.161397,0.003604,0.000016,0.000091


In [21]:
# find the columns that we need to synthesis data for to produce an attack dataset
columnsToGather = dosSamples.replace(0, np.nan) #replace all 0 values with null
columnsToGather = columnsToGather.dropna(how="all", axis=1).columns.tolist() #remove all columns where there are null values
columnsToGather #left with all columns that the values are not 0 (be know for a fact that the data is consistant and there are not missing values in the rows)

['Number of Ports',
 'SYN Flag Count',
 'Flow Duration',
 'Packets Per Second',
 'IAT Total',
 'IAT Max',
 'IAT Mean',
 'IAT Std']

In [23]:
# find an approximate minimum and maximum values of each column and save that data into a dictionary
MinMaxDict = {col: (dosSamples[col].min() * 0.85, dosSamples[col].max() * 1.15) for col in columnsToGather}
MinMaxDict['Number of Ports'] = (1, 1)
MinMaxDict

{'Number of Ports': (1, 1),
 'SYN Flag Count': (1343.0, 11500.0),
 'Flow Duration': (0.027222955226898115, 0.26944954395294185),
 'Packets Per Second': (36009.2092109628, 111222.937044877),
 'IAT Total': (0.027222955226898115, 0.26944954395294185),
 'IAT Max': (0.0013685345649718799, 0.021685552597045805),
 'IAT Mean': (8.789533757605497e-06, 2.714856866024604e-05),
 'IAT Std': (5.752392659748788e-05, 0.000393180927543065)}

In [24]:
# change values to int for the columns that should be int (because when calculating approximate values they will be turned into floats)
intColumns = ['SYN Flag Count']
for key, val in MinMaxDict.items():
    if key in intColumns:
        MinMaxDict[key] = tuple([int(v) for v in val])
MinMaxDict

{'Number of Ports': (1, 1),
 'SYN Flag Count': (1343, 11500),
 'Flow Duration': (0.027222955226898115, 0.26944954395294185),
 'Packets Per Second': (36009.2092109628, 111222.937044877),
 'IAT Total': (0.027222955226898115, 0.26944954395294185),
 'IAT Max': (0.0013685345649718799, 0.021685552597045805),
 'IAT Mean': (8.789533757605497e-06, 2.714856866024604e-05),
 'IAT Std': (5.752392659748788e-05, 0.000393180927543065)}

In [130]:
# creating an empty dataframe before adding values to it
dosDataset = pd.DataFrame(np.zeros((NUM_OF_ROWS, len(dosSamples.columns))), columns=dosSamples.columns)
dosDataset.head(3)

,Number of Ports,Average Packet Size,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,...,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Total,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [131]:
# adding zeros to all columns that should not have any values
zeroColumns = [col for col in dosSamples.columns if col not in columnsToGather]
for col in zeroColumns:
    dosDataset[col] = int(0)
zeroColumns

['Average Packet Size',
 'Packet Length Min',
 'Packet Length Max',
 'Packet Length Mean',
 'Packet Length Std',
 'Packet Length Variance',
 'Total Length of Fwd Packet',
 'Fwd Packet Length Max',
 'Fwd Packet Length Mean',
 'Bwd Packet Length Max',
 'Bwd Packet Length Mean',
 'Bwd Packet Length Min',
 'Bwd Packet Length Std',
 'Fwd Segment Size Avg',
 'Bwd Segment Size Avg',
 'Subflow Fwd Bytes',
 'ACK Flag Count',
 'RST Flag Count']

In [132]:
dosDataset.head(3)

,Number of Ports,Average Packet Size,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,...,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Total,IAT Max,IAT Mean,IAT Std
0,0.0,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [133]:
# finding the correlation between the Number of Ports column to the rest of the columns in order to create new data

independent_col = dosSamples[columnsToGather[1]].values.reshape(-1, 1) #column 'Number of Ports'
dependent_cols = dosSamples[columnsToGather[2:]].values #the rest of the columns that are not zeros

# find the scaling factors using least squares function
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond=None)[0]

scaling_factors = [(name,factor) for name, factor in zip(columnsToGather[2:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('Flow Duration', 1.5761780524472803e-05)
('Packets Per Second', 7.0046558779643675)
('IAT Total', 1.5761780524472803e-05)
('IAT Max', 6.217951120191447e-07)
('IAT Mean', 1.7101155275277512e-09)
('IAT Std', 1.274922396610739e-08)


In [134]:
# adding the rest of the attack feature values to the dataset at random based on the smaple data
dosDataset['Number of Ports'] = np.full(shape=NUM_OF_ROWS, fill_value=1, dtype=int)
dosDataset['SYN Flag Count'] = np.random.randint(MinMaxDict['SYN Flag Count'][0]*0.9, MinMaxDict['SYN Flag Count'][1]*1.05, NUM_OF_ROWS)

for index, row in dosDataset.iterrows():
    i = 0
    for col, factor in scaling_factors: #iterating over all rows we need to add values to except 'Number of Ports'
        # calculate a random small delta of the factor for adding some randomness
        if col in ['Packets Per Second', 'IAT Max']:
            delta = random.uniform(factor * 0.2, factor * 0.45) # select a dantom
        else:
            delta = random.uniform(factor * 0.1, factor * 0.2) # select a dantom

        # apply the randomness to the calculated number
        if col == 'IAT Max':
            updatedFactor = factor + random.choices([-1, 1], weights=[2, 1], k=1)[0] * delta
        else:
            updatedFactor = factor + random.choice([-1, 1]) * delta

        # calculate the value we want to add into the dataset in the given row
        dosDataset.loc[index, col] = row['SYN Flag Count'] * updatedFactor
        i+=1

In [135]:
# adding a label to the dataset
dosDataset['Label'] = ATTACK_NAME

In [136]:
dosDataset

,Number of Ports,Average Packet Size,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,...,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Total,IAT Max,IAT Mean,IAT Std,Label
0,1,0,0,0,0,0,0,0,0,0,...,11143,0,0,0.210407,52350.008012,0.203293,0.004944,0.000017,0.000116,DoS
1,1,0,0,0,0,0,0,0,0,0,...,2652,0,0,0.034620,14056.721642,0.047585,0.001158,0.000004,0.000028,DoS
2,1,0,0,0,0,0,0,0,0,0,...,2761,0,0,0.035557,12047.220322,0.050298,0.001092,0.000004,0.000041,DoS
3,1,0,0,0,0,0,0,0,0,0,...,4119,0,0,0.074213,21023.030895,0.075182,0.001775,0.000008,0.000061,DoS
4,1,0,0,0,0,0,0,0,0,0,...,8303,0,0,0.111533,37215.961745,0.117213,0.006409,0.000012,0.000122,DoS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,1,0,0,0,0,0,0,0,0,0,...,2034,0,0,0.037179,7915.715567,0.027342,0.000699,0.000003,0.000022,DoS
24996,1,0,0,0,0,0,0,0,0,0,...,8011,0,0,0.142775,73592.862824,0.144042,0.002848,0.000011,0.000089,DoS
24997,1,0,0,0,0,0,0,0,0,0,...,10377,0,0,0.131500,51084.058057,0.141591,0.008411,0.000020,0.000106,DoS
24998,1,0,0,0,0,0,0,0,0,0,...,3654,0,0,0.068242,15752.557147,0.051195,0.001339,0.000007,0.000056,DoS


In [138]:
x = dosDataset[dosDataset['SYN Flag Count'] > 9000]
x = x[x['SYN Flag Count'] < 10000]
x[0:30]

,Number of Ports,Average Packet Size,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,...,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Total,IAT Max,IAT Mean,IAT Std,Label
21,1,0,0,0,0,0,0,0,0,0,...,9792,0,0,0.180474,39190.766045,0.182088,0.008487,0.000014,0.000144,DoS
22,1,0,0,0,0,0,0,0,0,0,...,9410,0,0,0.126070,49460.726396,0.132510,0.004022,0.000014,0.000143,DoS
40,1,0,0,0,0,0,0,0,0,0,...,9422,0,0,0.166284,45677.106470,0.131337,0.003498,0.000013,0.000134,DoS
42,1,0,0,0,0,0,0,0,0,0,...,9678,0,0,0.136369,38900.702931,0.175239,0.003993,0.000014,0.000110,DoS
45,1,0,0,0,0,0,0,0,0,0,...,9801,0,0,0.172123,39079.048285,0.131641,0.003777,0.000019,0.000146,DoS
55,1,0,0,0,0,0,0,0,0,0,...,9267,0,0,0.168232,91453.634106,0.164820,0.007848,0.000019,0.000137,DoS
118,1,0,0,0,0,0,0,0,0,0,...,9193,0,0,0.121878,50776.745368,0.128136,0.003245,0.000013,0.000130,DoS
135,1,0,0,0,0,0,0,0,0,0,...,9455,0,0,0.119505,49194.017204,0.128299,0.003783,0.000013,0.000104,DoS
144,1,0,0,0,0,0,0,0,0,0,...,9575,0,0,0.176851,51435.711530,0.168351,0.004584,0.000020,0.000135,DoS
169,1,0,0,0,0,0,0,0,0,0,...,9647,0,0,0.169056,37617.377099,0.123431,0.007451,0.000013,0.000101,DoS


In [139]:
dosDataset.describe()

,Number of Ports,Average Packet Size,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,...,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Total,IAT Max,IAT Mean,IAT Std
count,25000.0,25000.0,25000.0,25000.0,25000.0,25000.0,25000.0,25000.0,25000.0,25000.0,...,25000.0,25000.000000,25000.0,25000.0,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000
mean,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6636.201200,0.0,0.0,0.104682,46609.754099,0.104651,0.003684,0.000011,0.000085
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3129.851931,0.0,0.0,0.052481,27803.530920,0.052420,0.002256,0.000006,0.000042
min,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1208.000000,0.0,0.0,0.015490,4720.310070,0.015410,0.000416,0.000002,0.000013
25%,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3920.000000,0.0,0.0,0.060577,24403.064673,0.060610,0.001956,0.000007,0.000049
50%,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6619.500000,0.0,0.0,0.102173,41527.123407,0.102371,0.003304,0.000011,0.000083
75%,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9338.250000,0.0,0.0,0.143819,62573.644502,0.143798,0.004748,0.000016,0.000116
max,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,12074.000000,0.0,0.0,0.228189,122265.907545,0.227687,0.010828,0.000025,0.000184


In [140]:
# save the dataset
# dosDataset.to_csv('dos_dataset_updated_flows.csv', index=False)

---